In [ ]:
import ee
import geemap
import os
import datetime # <<< FIX: Import Python's datetime library

# --- 1. INITIALIZATION & SETUP ---
PROJECT_ID = 'flash-griffin-473118-e3'

# Define the region of interest
CENTER_LON = -119.5
CENTER_LAT = 36.5
ZOOM_LEVEL = 6
FILTER_POINT = ee.Geometry.Point(CENTER_LON, CENTER_LAT)
BOUNDING_BOX = ee.Geometry.BBox(
    CENTER_LON - 5, CENTER_LAT - 5, CENTER_LON + 5, CENTER_LAT + 5
)

try:
    ee.Initialize(project=PROJECT_ID)
    print("✅ Earth Engine client initialized successfully.")
except Exception as e:
    print(f"❌ Error during initialization. Check ID and API enablement. Error: {e}")
    raise

# Create an interactive map
Map = geemap.Map(center=[CENTER_LON, CENTER_LAT], zoom=ZOOM_LEVEL)
Map.add_basemap('SATELLITE')

# ----------------------------------------------------------------------
# --- 2. DATA ACQUISITION & FILTERING (FOR LIVE DATA) ---
# ----------------------------------------------------------------------
METHANE_COLLECTION = 'COPERNICUS/S5P/OFFL/L3_CH4'

# Set a rolling date range for the last 7 days
DAYS_TO_VIEW = 7
END_DATE = ee.Date(datetime.datetime.now()) # <<< FIX: Use Python's syntax
START_DATE = END_DATE.advance(-DAYS_TO_VIEW, 'day')

# Note: S5P data has a few days of latency, so this will get the most recent *available* data.
methane_data = ee.ImageCollection(METHANE_COLLECTION) \
    .select('CH4_column_volume_mixing_ratio_dry_air') \
    .filterDate(START_DATE, END_DATE) \
    .filterBounds(FILTER_POINT) \
    .mean()

# ----------------------------------------------------------------------
# --- 3. VISUALIZATION OF METHANE SPOTS ---
# ----------------------------------------------------------------------

# This threshold hides the very low, background emissions so we only see the spots.
BACKGROUND_THRESHOLD = 1850

# This is the concentration that will get the brightest red color.
PEAK_THRESHOLD = 1950

# Apply the mask to make background pixels transparent
methane_spots = methane_data.updateMask(methane_data.gt(BACKGROUND_THRESHOLD))

# Define a palette that goes from a darker, faint red to a bright, solid red.
spot_style = {
    'min': BACKGROUND_THRESHOLD,
    'max': PEAK_THRESHOLD,
    'palette': ['#8B0000', '#FF0000'] # Dark Red to Bright Red
}

# Add the styled methane spots to the map
Map.addLayer(
    methane_spots,
    spot_style,
    f'Methane Hotspots (Last {DAYS_TO_VIEW} Days)'
)

# Display the map
Map